My solution to https://adventofcode.com/2023/day/11.

<ins>Part1</ins> is solved following the process described in the AoC text. The universe or mtx (for matrix) is represented as a list of strings. 
- `expand_universe` duplicates rows and columns that are entirely made of '.' representing empty space.
- `get_galaxies` returns the coordinates of every '#' representing galaxies.
- `sum_distances` returns the total of x,y distances between all pairs of galaxies based on the list of coordinates. 

<ins>Part2</ins>, as AoC tends to do, shows that the <ins>Part1</ins> solution is not scalable. Rows and columns represnting empty space are increased a million-fold instead of being duplicated. Instead of expanding the mtx object representing the universe, the solution keeps track of rows and columns that are to be replaced by a million (1000000) rows or columns.
- `get_galaxies` from Part1 is applied to the original mtx to generate the coordinates of the galaxies.
- `track_expansions` returns the rows and columns of the mtx that will be increased a million-fold.
- `sum_distances_P2` returns the total x,y distances between all pairs of galaxies while adding 999999 (a million minus one) for every empty row or column between individual pairs of galaxies.


In [1]:
#open file as df
import pandas as pd

filename = 'day11_input.txt'
with open (filename,'r') as f:
    mtx = f.read().splitlines()

print(f'The universe is a matrix of {len(mtx)} rows and {len(mtx[0])} columns.')

The universe is a matrix of 140 rows and 140 columns.


In [2]:
#Part 1 Functions

def expand_universe(mtx_orig: list[str]) -> list[str]:
    '''
    Args:
        mtx_orig : matrix as list of strings
    Returns:
        mtx : empty rows and columns of the original are duplicated, a longer list of longer strings
    '''
    mtx = mtx_orig.copy()
    empty_row = '.' * len(mtx[0])

    for row in range(len(mtx)-1,-1,-1):
        if mtx[row] == empty_row:
            mtx.insert(row,empty_row)

    totcols = len(mtx[0])
    totrows = len(mtx) #this has expanded

    for col in range(totcols-1,-1,-1):
        if all([mtx[row][col] == '.' for row in range(totrows)]):
            for row in range(totrows):
                mtx[row] = mtx[row][:col] + '.' + mtx[row][col:] 
    return mtx

def get_galaxies(mtx:list) -> list:
    '''
    Args:
        mtx : matrix as list of strings
    Returns:
        coordinates : list of (row,col) coordinates of '#' representing a galaxy
    '''
    coordinates = []
    for row in range(len(mtx)):
        for col in range(len(mtx[0])):
            if mtx[row][col] == '#':
                coordinates.append((row,col))
    return coordinates

def sum_distances(coordinates:list) -> int:
    '''
    Args:
        coordinates: list of (row,col) coordinates
    Returns: 
        total_distance : integer, sum of the distance (row difference + col difference) between every pair of coordinates
    '''
    total_distance = 0
    for i in range(len(coordinates)-1):
        for j in range(i+1,len(coordinates)):
            yi,xi = coordinates[i]
            yj,xj = coordinates[j]
            distance = abs(xj-xi) + abs(yj-yi)
            total_distance += distance

    return total_distance

    

In [3]:
#Part 1 Solution
mtx_expanded = expand_universe(mtx)
galaxies = get_galaxies(mtx_expanded)
total_distance = sum_distances(galaxies)
print(f'Part1 Answer: {total_distance}')

Part1 Answer: 9556896


In [4]:
#Part 2 Functions
def track_expansions(mtx:list) -> (list[int], list[int]):
    '''Args:
        mtx: matrix as list of strings
    Returns: 
        empty_rows: rows with only '.' 
        empty_cols: columns with only '.'
    '''
    totrows = len(mtx)
    totcols = len(mtx[0])
    empty_rows = [row for row in range(totrows) if set(mtx[row]) == {'.'}]
    empty_cols = [col for col in range(totcols) if all([mtx[row][col] == '.' for row in range(totrows)])]
    return empty_rows, empty_cols

def sum_distances_P2(coordinates: list, empty_rows: list, empty_cols: list, multiplier: int) -> int:
    '''
    Args:
        coordinates : list of (row,col) coordinates
        empty_rows : list of rows with only '.', these are replaced by empty rows * multiplier
        empty_cols: list of columns with only '.', these are replaced by empty columns * multiplier
        multiplier : int
    Returns: 
        total_distance : integer, sum of the distance (row difference + col difference) between 
            every pair of coordinates while adding an extra value (multiplier - 1) for every 
            empty row or column between coordinates
    '''
    mult = multiplier-1
    total_distance = 0
    for i in range(len(coordinates)-1):
        for j in range(i+1,len(coordinates)):
            yi,xi = coordinates[i]
            yj,xj = coordinates[j]
            x_distance = abs(xj-xi) + mult*sum([col in range(min(xi,xj),max(xi,xj)) for col in empty_cols])
            y_distance = abs(yj-yi) + mult*sum([row in range(min(yi,yj),max(yi,yj)) for row in empty_rows])
            total_distance += x_distance + y_distance
    return total_distance
    


In [5]:
#Part 2 Test with AoC example gives expected values
samplefile = 'day11_sample.txt'
with open (samplefile,'r') as f:
    stx = f.read().splitlines()
    
s_galaxies = get_galaxies(stx)
s_empty_rows, s_empty_cols = track_expansions(stx)
ten_dist = sum_distances_P2(s_galaxies, s_empty_rows, s_empty_cols, 10)
hundred_dist = sum_distances_P2(s_galaxies, s_empty_rows, s_empty_cols, 100)
print(f'example with multiplier 10: {ten_dist}')
print(f'example with multiplier 100: {hundred_dist}')


example with multiplier 10: 1030
example with multiplier 100: 8410


In [7]:
#Part 2 Solution
galaxies_p2 = get_galaxies(mtx)
empty_rows, empty_cols = track_expansions(mtx)
sum_distances_1000000 = sum_distances_P2(galaxies_p2, empty_rows, empty_cols, 1000000)

print(f'Part2 Answer: {sum_distances_1000000}')

Part2 Answer: 685038186836
